<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/papers/Artigo%201/V3/Vers%C3%A3o_3_(ajustes_de_hiperpar%C3%A2metros_SVM).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from sklearn.svm import SVC
from concurrent.futures import ProcessPoolExecutor

In [2]:
# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/Reprodu-o/main/export.csv')

# Verificar e remover valores faltantes
print("Valores faltantes por coluna:\n", df.isnull().sum())
df.dropna(inplace=True)

# Definir variáveis preditoras (X) e alvo binária (y)
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis',
        'Origin recode NHIA (Hispanic, Non-Hisp)']]
y = (df['SEER Combined Mets at DX-liver (2010+)'] == 'Yes') | (df['SEER Combined Mets at DX-lung (2010+)'] == 'Yes')
y = y.astype(int)

Valores faltantes por coluna:
 Patient ID                                         0
Age recode with <1 year olds                       0
Sex                                                0
Race recode (White, Black, Other)                  0
Histologic Type ICD-O-3                            0
Grade Recode (thru 2017)                           0
Primary Site                                       0
Derived AJCC T, 7th ed (2010-2015)                 0
Derived AJCC N, 7th ed (2010-2015)                 0
CS tumor size (2004-2015)                          0
CEA Pretreatment Interpretation Recode (2010+)     0
Tumor Deposits Recode (2010+)                      0
Marital status at diagnosis                        0
Origin recode NHIA (Hispanic, Non-Hisp)            0
SEER Combined Mets at DX-lung (2010+)             15
SEER Combined Mets at DX-liver (2010+)            12
SEER Combined Mets at DX-bone (2010+)             14
ICD-O-3 Hist/behav                                 0
ICD-O-3 Hist/be

In [3]:
from concurrent.futures import ProcessPoolExecutor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import roc_auc_score, classification_report

# Função para realizar busca em um bloco
def fit_block(n_iter, block_index, X_train, y_train, param_grid, cv):
    """
    Executa RandomizedSearchCV em um bloco de dados.
    """
    print(f"\nIniciando bloco {block_index + 1} com {n_iter} iterações...")
    random_search = RandomizedSearchCV(
        estimator=SVC(probability=True),
        param_distributions=param_grid,
        n_iter=n_iter,
        scoring="roc_auc",
        cv=cv,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )
    random_search.fit(X_train, y_train)
    print(f"Bloco {block_index + 1} concluído.")
    return random_search.best_params_, random_search.best_score_

# Divisão em blocos e paralelização
def parallel_random_search(blocks, block_size, X_train, y_train, param_grid, cv):
    """
    Paraleliza a execução de RandomizedSearchCV em blocos.
    """
    with ProcessPoolExecutor() as executor:
        futures = [
            executor.submit(fit_block, block_size, i, X_train, y_train, param_grid, cv)
            for i in range(blocks)
        ]
        results = [future.result() for future in futures]
    return results

# Ajuste de hiperparâmetros
def tune_hyperparameters(X_train, y_train, X_test, y_test, param_grid, cv, blocks=3, block_size=10):
    """
    Realiza ajuste de hiperparâmetros em blocos e avalia o melhor modelo no conjunto de teste.
    """
    # Realizar busca paralela
    results = parallel_random_search(blocks, block_size, X_train, y_train, param_grid, cv)

    # Exibir resultados de cada bloco
    for i, (params, score) in enumerate(results):
        print(f"\nBloco {i + 1} - Melhores parâmetros: {params}")
        print(f"Bloco {i + 1} - Melhor pontuação (AUC-ROC): {score:.4f}")

    # Obter os melhores parâmetros do último bloco
    best_params = results[-1][0]
    best_model = SVC(**best_params, probability=True).fit(X_train, y_train)

    # Avaliação do modelo no conjunto de teste
    y_pred_proba = best_model.predict_proba(X_test)[:, 1]
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    y_pred = best_model.predict(X_test)

    print(f"\nAUC-ROC no conjunto de teste: {roc_auc:.4f}")
    print("\nRelatório de classificação no conjunto de teste:")
    print(classification_report(y_test, y_pred))

    return best_model, roc_auc

# Parâmetros do modelo e da validação
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf', 'poly'],
    'gamma': ['scale', 'auto'],
    'degree': [2, 3, 4]
}
cv = 3  # Número de folds para validação cruzada

# Simulação dos dados (substituir pelos seus conjuntos reais)
# X_train_bal, y_train_bal, X_test, y_test são seus datasets pré-processados
import numpy as np
X_train_bal = np.random.rand(100, 5)
y_train_bal = np.random.randint(0, 2, 100)
X_test = np.random.rand(30, 5)
y_test = np.random.randint(0, 2, 30)

# Número de blocos e iterações por bloco
blocks = 3
block_size = 10

# Execução do ajuste de hiperparâmetros e avaliação
best_model, test_roc_auc = tune_hyperparameters(
    X_train_bal, y_train_bal, X_test, y_test, param_grid, cv, blocks, block_size
)



Iniciando bloco 1 com 10 iterações...
Iniciando bloco 2 com 10 iterações...

Fitting 3 folds for each of 10 candidates, totalling 30 fitsFitting 3 folds for each of 10 candidates, totalling 30 fits



/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Bloco 1 concluído.

Iniciando bloco 3 com 10 iterações...
Bloco 2 concluído.
Fitting 3 folds for each of 10 candidates, totalling 30 fits
Bloco 3 concluído.

Bloco 1 - Melhores parâmetros: {'kernel': 'poly', 'gamma': 'scale', 'degree': 3, 'C': 10}
Bloco 1 - Melhor pontuação (AUC-ROC): 0.6957

Bloco 2 - Melhores parâmetros: {'kernel': 'poly', 'gamma': 'scale', 'degree': 3, 'C': 10}
Bloco 2 - Melhor pontuação (AUC-ROC): 0.6957

Bloco 3 - Melhores parâmetros: {'kernel': 'poly', 'gamma': 'scale', 'degree': 3, 'C': 10}
Bloco 3 - Melhor pontuação (AUC-ROC): 0.6957

AUC-ROC no conjunto de teste: 0.7200

Relatório de classificação no conjunto de teste:
              precision    recall  f1-score   support

           0       0.73      0.53      0.62        15
           1       0.63      0.80      0.71        15

    accuracy                           0.67        30
   macro avg       0.68      0.67      0.66        30
weighted avg       0.68      0.67      0.66        30

